In [1]:
import cv2
import numpy as np

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
%cd /content/drive/MyDrive/Robotics

/content/drive/MyDrive/Robotics


In [14]:
yolo_config_path = "/content/drive/MyDrive/Roboticsruns/detect/train5/argv.yaml"
yolo_weights_path = "/content/drive/MyDrive/Roboticsruns/detect/train5/weights/best.pt"
names_path = "/content/drive/MyDrive/Robotics/data.yaml"

In [15]:
def load_yolo():
    net = cv2.dnn.readNet(yolo_weights_path, yolo_config_path)
    classes = []
    with open(names_path, "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layers_names = net.getUnconnectedOutLayersNames()
    return net, classes, layers_names

In [16]:
def object_detection(frame, net, layers_names, confidence_threshold=0.5):
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(layers_names)

    detected_objects = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                detected_objects.append((classes[class_id], confidence, (x, y, w, h)))

    return detected_objects

In [17]:
def main():
    # Load YOLO model
    net, classes, layers_names = load_yolo()

    # Open a connection to the webcam (0 represents the default camera)
    cap = cv2.VideoCapture(0)

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Perform object detection
        detected_objects = object_detection(frame, net, layers_names)

        # Draw bounding boxes and display results
        for obj in detected_objects:
            label, confidence, box = obj
            x, y, w, h = box
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{label}: {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the resulting frame
        cv2.imshow("Object Detection", frame)

        # Break the loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything is done, release the capture
    cap.release()
    cv2.destroyAllWindows()

In [20]:
main()

error: OpenCV(4.8.0) /io/opencv/modules/dnn/src/dnn_read.cpp:56: error: (-2:Unspecified error) Cannot determine an origin framework of files: /content/drive/MyDrive/Roboticsruns/detect/train5/weights/best.pt, /content/drive/MyDrive/Roboticsruns/detect/train5/argv.yaml in function 'readNet'


In [21]:
import cv2
from pathlib import Path

# Load YOLO model
model_path = Path("runs/detect/train5/weights/best.pt")
net = cv2.dnn.readNet(str(model_path))

# Load class names
class_names = ["Robot", "Target"]

# Open camera
cap = cv2.VideoCapture(0)  # Use 0 for default camera, you can change it if you have multiple cameras

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Prepare the frame for YOLO
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (224, 224), (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)

    # Perform object detection
    detections = net.forward()

    # Loop over the detections
    for detection in detections[0, 0]:
        confidence = detection[2]
        if confidence > 0.5:  # You can adjust the confidence threshold
            class_id = int(detection[1])
            class_name = class_names[class_id]
            box = detection[3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype("int")

            # Draw bounding box and label on the frame
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, class_name, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Object Detection", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.8.0) /io/opencv/modules/dnn/src/dnn_read.cpp:56: error: (-2:Unspecified error) Cannot determine an origin framework of files: runs/detect/train5/weights/best.pt in function 'readNet'
